In [ ]:
# Colab smoke test: load this repo and import your modules
# Option A (recommended): clone your GitHub repo
#   1) Set REPO_URL to your repository URL
#   2) Run this cell

REPO_URL = "https://github.com/<user>/<repo>.git"  # TODO: replace
REPO_DIR = "DataAgent"  # folder name after clone (adjust if different)

import os, sys

# If you're running this notebook *inside the repo already* (e.g., Drive mount),
# set REPO_URL = "" and set REPO_DIR = "/content/drive/MyDrive/<path>/DataAgent".

if REPO_URL:
    !git clone -q {REPO_URL} {REPO_DIR}

%cd {REPO_DIR}

# Install dependencies
!pip -q install -r requirements.txt

# Make repo importable
sys.path.insert(0, os.getcwd())

print("Repo cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

# Smoke-test imports (no model call yet)
import Agent
from Agent import data_agent
from Agent.data_agent import SalesDataAgent

print("Imported OK")
print("Agent package:", Agent.__file__)
print("SalesDataAgent:", SalesDataAgent)

# Optional: check LangGraph import (used by the agent)
import langgraph
print("langgraph:", getattr(langgraph, "__version__", "(no __version__)"))


In [ ]:
# If you want to *run* the agent in Colab, you need an LLM backend accessible from Colab.
# Ollama is local to your PC, so it won't work in Colab unless you expose it.
#
# Example placeholder (will fail unless you configure a remote backend):
# agent = SalesDataAgent(model="llama3.2:3b", ollama_url="http://localhost:11434")
# agent.run("What is the total sales in october 2022?", no_vis=True, save_dir="./output_colab")

print("Import test complete. Configure an LLM backend to run the agent.")


In [ ]:
# Run a YAML experiment in Colab (like you do locally)
# This builds the Linux C++ runner, then runs it with a Colab-safe YAML copy.

# 1) Pick which YAML you want to run
CONFIG_PATH = "config/agent_config.yaml"  # or "config/agent_config_exp1_bestof_sweep.yaml"

# 2) Install build tools + YAML parser
!apt-get -qq update
!apt-get -qq install -y build-essential cmake
!pip -q install pyyaml

# (Optional) Check Java availability (needed for SPICE)
!java -version

# 3) Build the C++ runner for Linux (Colab)
!cmake -S my_cpp -B my_cpp/build-colab -DCMAKE_BUILD_TYPE=Release
!cmake --build my_cpp/build-colab --target agent_config_runner -j 2

# 4) Create a Colab-safe YAML copy (so you don't need to edit your repo YAML)
import yaml
from pathlib import Path

cfg = yaml.safe_load(Path(CONFIG_PATH).read_text(encoding="utf-8"))

# Colab uses linux python
cfg["python_bin"] = "python3"

# IMPORTANT: Ollama runs on your local PC, so it won't be reachable from Colab.
# If you keep Ollama, this will fail unless you provide a reachable ollama_url.
# Example (ONLY if you have a public endpoint / tunnel):
# cfg["ollama_url"] = "http://<public-host>:11434"

# SPICE: point the YAML-copy to a jar inside the repo if available.
# Recommended: include the folder SPICE-1.0/ in your repo or place it in the Colab workspace.
spice_jar = Path("SPICE-1.0/spice-1.0.jar")
if spice_jar.exists():
    cfg.setdefault("spice", {})
    cfg["spice"]["jar_path"] = str(spice_jar)
    cfg["spice"]["java_bin"] = "java"
    print("Using SPICE jar:", spice_jar)
else:
    print("SPICE jar not found at", spice_jar)
    print("Place it at SPICE-1.0/spice-1.0.jar (e.g., add to repo or download in Colab).")

# If you just want to test the runner wiring without model calls, set sweep.enabled=false and run_batch=false,
# then use a very small prompt and expect the LLM call to be the only blocker.

tmp_yaml = Path("/content/agent_config_colab.yaml")
with tmp_yaml.open("w", encoding="utf-8") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Wrote:", tmp_yaml)

# 5) Run the YAML via the runner
!./my_cpp/build-colab/agent_config_runner {tmp_yaml}
